In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate()

sc= spark.sparkContext
#RDD
dataRDD= sc.parallelize([("Brooke",20),("Denny",31),("Jules",30),("TD",35),("Brooke",25),("Jules",30),("Denny",51),("TD",15)])

dataRDD.collect()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/19 01:36:26 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/10/19 01:36:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/19 01:36:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('Brooke', 20),
 ('Denny', 31),
 ('Jules', 30),
 ('TD', 35),
 ('Brooke', 25),
 ('Jules', 30),
 ('Denny', 51),
 ('TD', 15)]

In [4]:
data_frame= spark.createDataFrame([("Brooke",20),("Denny",31),("Jules",30),("TD",35),
("Brooke",25),("Jules",30),("Denny",51),("TD",15)],["name","age"])

#data_frame.printSchema()
#data_frame.show()  
data_frame.show(5)  
r=data_frame.collect()[0]
r.name

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
+------+---+
only showing top 5 rows


'Brooke'

In [5]:
# استخدام group by عشان اعمل متوسط عن طريق العمر 
 
avg_age=data_frame.groupBy('name').avg('age')
avg_age.show()

# عشان اعرف عدد البرتشننز الي هو 1 بس 
avg_age.rdd.getNumPartitions() # فوق كان في 2 بارتشن بس هنا 1 بس لانه عمل جروب باي يعني عمل شفل بين الي فوق عشان يطلع ببرتشن واحد بس 

avg_age.head(2)


+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Jules|    30.0|
|    TD|    25.0|
| Denny|    41.0|
+------+--------+



[Row(name='Brooke', avg(age)=22.5), Row(name='Jules', avg(age)=30.0)]

In [31]:
#يعني اي agg 
data = [
    ("Ahmed", "IT", 3000),
    ("Sara", "IT", 4000),
    ("Ali", "HR", 2000),
    ("Mona", "HR", 2500)
]

columns = ["name", "department", "salary"]

df = spark.createDataFrame(data, columns)
df.show()


# تجميع بدون group by
from pyspark.sql import functions as F

df.agg(
    F.sum("salary").alias("total_salary"),
    F.avg("salary").alias("avg_salary")
).show()


# تجميع مع group by 
df.groupBy("department").agg(
    F.sum("salary").alias("total_salary"),
    F.avg("salary").alias("avg_salary")
).show()



+-----+----------+------+
| name|department|salary|
+-----+----------+------+
|Ahmed|        IT|  3000|
| Sara|        IT|  4000|
|  Ali|        HR|  2000|
| Mona|        HR|  2500|
+-----+----------+------+



+------------+----------+
|total_salary|avg_salary|
+------------+----------+
|       11500|    2875.0|
+------------+----------+



+----------+------------+----------+
|department|total_salary|avg_salary|
+----------+------------+----------+
|        IT|        7000|    3500.0|
|        HR|        4500|    2250.0|
+----------+------------+----------+



git data from json file 
--------------------------

In [ ]:
df=spark.read.json('/home/ahmed-refat/Downloads/people.json')

df.show()
name=df['name']
# استخدام sql statments
df.select('name').show()

+--------------------+----+-------+
|     _corrupt_record| age|   name|
+--------------------+----+-------+
|[ {"name": "ahmed...|NULL|   NULL|
|                NULL|  34|mohamed|
|                NULL|  22|   sara|
+--------------------+----+-------+

+-------+
|   name|
+-------+
|   NULL|
|mohamed|
|   sara|
+-------+



+-------+
|   name|
+-------+
|   NULL|
|mohamed|
|   sara|
+-------+

